# Data Exploration 

In [50]:
import numpy as np
import pandas as pd 
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Connect to Local SQL Database

In [11]:
conn = sqlite3.connect("data/im.db")
imdb_people = pd.read_sql( 
    """
    SELECT ordering, movie_basics.start_year, movie_basics.primary_title, movie_ratings.averagerating, persons.primary_name, genres, characters, category, principals.job 
        FROM principals 
            JOIN persons
                ON principals.person_id = persons.person_id
            JOIN movie_basics 
                ON principals.movie_id = movie_basics.movie_id
            JOIN movie_ratings
                ON principals.movie_id = movie_ratings.movie_id
    """
, conn
)

#standardize column names
imdb_people = imdb_people.rename(columns={"primary_title": "title", "start_year": "year"})
imdb_people.head()

,ordering,year,title,averagerating,primary_name,genres,characters,category,job
0,10,2011,The Wicker Tree,3.9,Sean Barton,"Drama,Horror",None,editor,None
1,1,2011,The Wicker Tree,3.9,Brittania Nicol,"Drama,Horror","[""Beth Boothby""]",actress,None
2,2,2011,The Wicker Tree,3.9,Henry Garrett,"Drama,Horror","[""Steve Thomson""]",actor,None
3,3,2011,The Wicker Tree,3.9,Graham McTavish,"Drama,Horror","[""Sir Lachlan Morrison""]",actor,None
4,4,2011,The Wicker Tree,3.9,Jacqueline Leonard,"Drama,Horror","[""Lady Delia Morrison""]",actress,None


In [13]:
imdb_movies = pd.read_sql( 
    """
    SELECT movie_basics.start_year, movie_basics.primary_title, movie_ratings.averagerating, genres 
        FROM movie_basics 
            JOIN movie_ratings
                ON movie_basics.movie_id == movie_ratings.movie_id
    """
, conn
)
imdb_movies = imdb_movies.rename(columns={"primary_title": "title", "start_year": "year", "averagerating": "rating"})
imdb_movies.head()

,year,title,rating,genres
0,2013,Sunghursh,7.0,"Action,Crime,Drama"
1,2019,One Day Before the Rainy Season,7.2,"Biography,Drama"
2,2018,The Other Side of the Wind,6.9,Drama
3,2018,Sabse Bada Sukh,6.1,"Comedy,Drama"
4,2017,The Wandering Soap Opera,6.5,"Comedy,Drama,Fantasy"


# CSV and TSV Imports

In [15]:
#https://www.boxofficemojo.com/
bom = pd.read_csv('data/bom.movie_gross.csv')
#https://www.themoviedb.org/
tmdb = pd.read_csv('data/tmdb.movies.csv')
#https://www.the-numbers.com/
tn_movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')
#Academy_Awards_DB_from_https://www.kaggle.com/datasets/unanimad/the-oscar-award
oscars = pd.read_csv('data/the_oscar_award.csv')

### CSV Head Checks + Basic Cleaning

We making sure each DataFrame is using a datetimes for dates and cleaning any 'unique' systems, like TMDB using a numeric code for genre. We also standardize column names. 

#### Cleaning Functions

In [17]:
def csStringToList(cs_string):
    if type(cs_string) == str:
        return cs_string.split(',')
    else:
        return cs_string
    
def money_to_int(money):
    if type(money) != str:
        return money
    if '$' in money:
        #remove cash symbol
        money = money.replace('$', '')
    money = money.replace(',', '')
    money.strip()
    return money

Make sure the IMDB dfs are also using lists of genres.

In [19]:
imdb_movies['genres'] = imdb_movies['genres'].map(csStringToList)
imdb_people['genres'] = imdb_people['genres'].map(csStringToList)

## Clean BOM

In [106]:
bom = bom.rename(columns={"foreign_gross": "worldwide_gross"})
bom['domestic_gross'] = bom['domestic_gross'].map(money_to_int).astype(float)
bom['worldwide_gross'] = bom['worldwide_gross'].map(money_to_int).astype(float)
bom['worldwide_gross'] =  bom['worldwide_gross'] + bom['domestic_gross']

bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            3387 non-null   object 
 1   studio           3382 non-null   object 
 2   domestic_gross   3359 non-null   float64
 3   worldwide_gross  2009 non-null   float64
 4   year             3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


In [104]:
bom['studio'].value_counts()

studio
IFC           166
Uni.          147
WB            140
Fox           136
Magn.         136
             ... 
E1              1
PI              1
ELS             1
PalT            1
Synergetic      1
Name: count, Length: 257, dtype: int64

## Clean TMDB

In [25]:
def genreIDtoGenre(id_list):
    #'comma separated' 
    cs = id_list[1:-1]
    cs.strip()
    ids = cs.split(', ')
    newlist = []
    for id in ids:
        if id == "12":
            newlist.append("Adventure")
        if id == "28":
            newlist.append("Action")
        if id == "16":
            newlist.append("Animation")
        if id == "35":
            newlist.append("Comedy")
        if id == "80":
            newlist.append("Crime")
        if id == "99":
            newlist.append("Documentary")
        if id == "18":
            newlist.append("Drama")
        if id == "10751":
            newlist.append("Family")
        if id == "14":
            newlist.append("Fantasy")
        if id == "36":
            newlist.append("History")
        if id == "27":
            newlist.append("Horror")
        if id == "10402":
            newlist.append("Music")
        if id == "9648":
            newlist.append("Mystery")
        if id == "10749":
            newlist.append("Romance")
        if id == "878":
            newlist.append("Science Fiction")
        if id == "10770":
            newlist.append("TV Movie")
        if id == "53":
            newlist.append("Thriller")
        if id == "10752":
            newlist.append("War")
        if id == "37":
            newlist.append("Western")
    return newlist

tmdb['genres'] = tmdb['genre_ids'].map(genreIDtoGenre)
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'])
tmdb['year'] = tmdb['release_date'].dt.year

tmdb = tmdb.rename(columns={"vote_average": "rating"})

tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,rating,vote_count,genres,year
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,"[Adventure, Fantasy, Family]",2010
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,"[Fantasy, Adventure, Animation, Family]",2010
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,"[Adventure, Action, Science Fiction]",2010
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,"[Animation, Comedy, Family]",1995
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,"[Action, Science Fiction, Adventure]",2010


### Clean TN

In [36]:
def checkUSD(money):
    money = money[0]
    if money =='$':
        return True
    else:
        return False

tn_movie_budgets['release_date'] = pd.to_datetime(tn_movie_budgets['release_date'])
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].map(money_to_int)
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].map(money_to_int)
tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].map(money_to_int)
tn_movie_budgets['year'] = tn_movie_budgets['release_date'].dt.year

tn_movie_budgets = tn_movie_budgets.rename(columns={"movie": "title"})

tn_movie_budgets.head()

,id,release_date,title,production_budget,domestic_gross,worldwide_gross,year
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017


# Feature Engineering 

The goals are to create two main data frames. We want one to center around movie titles, studios, important people, production budgets, etc. The other will include genre information and public reception.

We can then merge these to have one main dataframe with the sum of our knowlegde

## Movie Data

We first merge bom and tn_movie budgets, as they are our main sources of financial data.

In [38]:
budget_info = pd.concat([tn_movie_budgets, bom])
budget_info['production_budget'] = budget_info['production_budget'].fillna(value=0)
budget_info['domestic_gross'] = budget_info['domestic_gross'].fillna(value=0)
budget_info['worldwide_gross'] = budget_info['worldwide_gross'].fillna(value=0)

budget_info['production_budget'] = budget_info['production_budget'].astype(float)
budget_info['domestic_gross'] = budget_info['domestic_gross'].astype(float)
budget_info['worldwide_gross'] = budget_info['worldwide_gross'].astype(float)

budget_info['domestic_financial_ratio'] = budget_info['production_budget'] / budget_info['domestic_gross']
budget_info['worldwide_financial_ratio'] = budget_info['production_budget'] / budget_info['worldwide_gross']

budget_info = budget_info.drop(columns='id')

budget_info.info()
budget_info.head()
budget_info.loc[budget_info['title'] == "Avatar"]

<class 'pandas.core.frame.DataFrame'>
Index: 9169 entries, 0 to 3386
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   release_date               5782 non-null   datetime64[ns]
 1   title                      9169 non-null   object        
 2   production_budget          9169 non-null   float64       
 3   domestic_gross             9169 non-null   float64       
 4   worldwide_gross            9169 non-null   float64       
 5   year                       9169 non-null   int64         
 6   studio                     3382 non-null   object        
 7   domestic_financial_ratio   9141 non-null   float64       
 8   worldwide_financial_ratio  7819 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(2)
memory usage: 716.3+ KB


,release_date,title,production_budget,domestic_gross,worldwide_gross,year,studio,domestic_financial_ratio,worldwide_financial_ratio
0,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2009,NaN,0.558837,0.153079


We then want to merge IMBD and TMDB, which have more information on ratings.

In [40]:
#joining TMDB and IMDB on the title and year
tmdb_imdb = pd.concat([imdb_movies, tmdb])

tmdb_imdb = tmdb_imdb.drop(columns=['genre_ids', 'id', 'Unnamed: 0'])
tmdb_imdb.info()
tmdb_imdb.head()

tmdb_imdb.loc[tmdb_imdb['title'] == "Avatar"]

<class 'pandas.core.frame.DataFrame'>
Index: 100373 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   year               100373 non-null  int64         
 1   title              100373 non-null  object        
 2   rating             100373 non-null  float64       
 3   genres             99569 non-null   object        
 4   original_language  26517 non-null   object        
 5   original_title     26517 non-null   object        
 6   popularity         26517 non-null   float64       
 7   release_date       26517 non-null   datetime64[ns]
 8   vote_count         26517 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 7.7+ MB


,year,title,rating,genres,original_language,original_title,popularity,release_date,vote_count
9944,2011,Avatar,6.1,[Horror],NaN,NaN,NaN,NaT,NaN
6,2009,Avatar,7.4,"[Action, Adventure, Fantasy, Science Fiction]",en,Avatar,26.526,2009-12-18,18676.0


## Big Merge
The movie DB is the combination of our ratings and our budget information.


In [100]:
movie_db = tmdb_imdb.merge(budget_info, how='left', on=['title', 'release_date'])

movie_db.info()
movie_db.head()

movie_db.loc[movie_db['title'] == "Avatar"]
movie_db.to_csv("movies_db.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100375 entries, 0 to 100374
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   year_x                     100375 non-null  int64         
 1   title                      100375 non-null  object        
 2   rating                     100375 non-null  float64       
 3   genres                     99571 non-null   object        
 4   original_language          26517 non-null   object        
 5   original_title             26517 non-null   object        
 6   popularity                 26517 non-null   float64       
 7   release_date               26517 non-null   datetime64[ns]
 8   vote_count                 26517 non-null   float64       
 9   production_budget          4427 non-null    float64       
 10  domestic_gross             4427 non-null    float64       
 11  worldwide_gross            4427 non-null    float64 

In [96]:
if movie_db['production_budget'] == 0 or movie_db['worldwide_gross'] == 0:
    movie_db['%_profit'] = np.nan()

else:
    movie_db['%_profit'] = (movie_db['worldwide_gross'] - movie_db['production_budget']) / movie_db['production_budget'] * 100

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [102]:
movie_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100375 entries, 0 to 100374
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   year_x                     100375 non-null  int64         
 1   title                      100375 non-null  object        
 2   rating                     100375 non-null  float64       
 3   genres                     99571 non-null   object        
 4   original_language          26517 non-null   object        
 5   original_title             26517 non-null   object        
 6   popularity                 26517 non-null   float64       
 7   release_date               26517 non-null   datetime64[ns]
 8   vote_count                 26517 non-null   float64       
 9   production_budget          4427 non-null    float64       
 10  domestic_gross             4427 non-null    float64       
 11  worldwide_gross            4427 non-null    float64 

In [86]:
movie_db['%_profit'].describe()

count    3232.000000
mean             inf
std              NaN
min      -100.000000
25%       166.972848
50%              NaN
75%              NaN
max              inf
Name: %_profit, dtype: float64

In [88]:
movie_db['production_budget'].describe()

count    4.427000e+03
mean     1.451949e+07
std      3.846886e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      7.000000e+06
max      4.250000e+08
Name: production_budget, dtype: float64

In [90]:
movie_db['worldwide_gross'].describe()

count    4.427000e+03
mean     7.928493e+07
std      1.735585e+08
min      0.000000e+00
25%      0.000000e+00
50%      1.040000e+07
75%      7.493343e+07
max      2.776345e+09
Name: worldwide_gross, dtype: float64

In [80]:
movie_db['%_profit'].value_counts().sort_index()

%_profit
-inf             1832
-4.155647e+04       1
-1.007595e+04       1
-8.556669e+03       1
-6.558059e+03       1
                 ... 
 9.988343e+01       1
 9.989182e+01       1
 9.989440e+01       1
 9.999740e+01       1
 1.000000e+02      42
Name: count, Length: 1231, dtype: int64

In [98]:
movie_db = movie_db.loc[(movie_db['worldwide_gross'] != 0) & (movie_db['production_budget'] != 0)]
movie_db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97306 entries, 0 to 100374
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   year_x                     97306 non-null  int64         
 1   title                      97306 non-null  object        
 2   rating                     97306 non-null  float64       
 3   genres                     96509 non-null  object        
 4   original_language          26475 non-null  object        
 5   original_title             26475 non-null  object        
 6   popularity                 26475 non-null  float64       
 7   release_date               26475 non-null  datetime64[ns]
 8   vote_count                 26475 non-null  float64       
 9   production_budget          1358 non-null   float64       
 10  domestic_gross             1358 non-null   float64       
 11  worldwide_gross            1358 non-null   float64       
 12  year_y  

In [108]:
oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10889 entries, 0 to 10888
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10889 non-null  int64 
 1   year_ceremony  10889 non-null  int64 
 2   ceremony       10889 non-null  int64 
 3   category       10889 non-null  object
 4   name           10884 non-null  object
 5   film           10570 non-null  object
 6   winner         10889 non-null  bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 521.2+ KB


In [138]:
oscars = oscars[oscars['film'].notna()]
oscars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10570 entries, 0 to 10884
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10570 non-null  int64 
 1   year_ceremony  10570 non-null  int64 
 2   ceremony       10570 non-null  int64 
 3   category       10570 non-null  object
 4   name           10570 non-null  object
 5   film           10570 non-null  object
 6   winner         10570 non-null  bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 588.4+ KB


In [110]:
oscars.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [112]:
oscars['category'].value_counts()

category
DIRECTING                                    469
FILM EDITING                                 450
ACTRESS IN A SUPPORTING ROLE                 440
ACTOR IN A SUPPORTING ROLE                   440
DOCUMENTARY (Short Subject)                  378
                                            ... 
SPECIAL FOREIGN LANGUAGE FILM AWARD            2
GORDON E. SAWYER AWARD                         1
SPECIAL ACHIEVEMENT AWARD (Sound Editing)      1
SPECIAL ACHIEVEMENT AWARD (Sound Effects)      1
AWARD OF COMMENDATION                          1
Name: count, Length: 115, dtype: int64

In [118]:
oscar_categ = oscars['category'].value_counts()

In [120]:
oscar_categ.to_csv('data/oscar_categ.csv')

In [136]:
oscars.loc[oscars['film'].isna(), ['name', 'year_film']]

,name,year_film
16,Ralph Hammeras,1927
18,Nugent Slaughter,1927
31,Joseph Farnham,1927
32,"George Marion, Jr.",1927
33,Warner Bros.,1927
...,...,...
10763,Iain Neil,2022
10885,NaN,2023
10886,"To Angela Bassett, who has inspired audiences ...",2023
10887,"To Mel Brooks, for his comedic brilliance, pro...",2023


In [140]:
oscar_categ_map = pd.read_csv('data/oscar_categ_map.csv')

In [142]:
oscar_categ_map.head()

,Major,Minor
0,ANIMATED FEATURE FILM,ACTOR
1,ASSISTANT DIRECTOR,ACTOR IN A LEADING ROLE
2,AWARD OF COMMENDATION,ACTOR IN A SUPPORTING ROLE
3,BEST MOTION PICTURE,ACTRESS
4,BEST PICTURE,ACTRESS IN A LEADING ROLE


In [144]:
major_oscars = oscar_categ_map['Major'].to_list()

In [148]:
minor_oscars = oscar_categ_map['Minor'].to_list()

In [ ]:
def winner_count(a, b, c):
    return a*b*c

# Using apply:
%timeit df['d'] = df.apply(lambda x: some_func(a = x['a'], b = x['b'], c = x['c']), axis=1)